<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%B8%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [62]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/14qT33yoGpMBe868XI6yvd8Hy7DldM4RhovGYNRtCUMk/edit?hl=zh-tw&gid=0#gid=0')

In [63]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('工作表1').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()


,日期,時間,品項,金額
0,2025/4/19,9:45,麵包,50
1,2025-04-19,9:45,麵包,50
2,2025/4/19,9:45,麵包,50
3,2025-04-19,9:45,麵包,50
4,2023-05-03,9:34,蛋餅,30


In [64]:
import datetime

In [65]:
# 讓使用者輸入資料
date_str = input("請輸入日期 (格式：YYYY-MM-DD): ")
# 檢查日期格式是否正確
datetime.datetime.strptime(date_str, '%Y-%m-%d')

time_str = input("請輸入時間 (格式：HH:MM): ")
# 檢查時間格式是否正確
datetime.datetime.strptime(time_str, '%H:%M')

item = input("請輸入品項: ")
amount = float(input("請輸入金額: "))
# 類別選單
categories = ["早餐", "午餐", "晚餐", "購物", "娛樂"]
print("\n請選擇類別：")
for i, cat in enumerate(categories, start=1):
    print(f"{i}. {cat}")

choice = int(input("輸入數字 (1-5): "))
category = categories[choice - 1]

請輸入日期 (格式：YYYY-MM-DD): 2023-03-10
請輸入時間 (格式：HH:MM): 08:23
請輸入品項: 餅乾
請輸入金額: 45

請選擇類別：
1. 早餐
2. 午餐
3. 晚餐
4. 購物
5. 娛樂
輸入數字 (1-5): 5


In [66]:
date_str

'2023-03-10'

In [67]:
time_str

'08:23'

In [68]:
item

'餅乾'

In [69]:
amount

45.0

In [70]:
category

'娛樂'

In [71]:
# 創建一個新的 DataFrame 來代表你要新增的資料
new_data = pd.DataFrame([{
    '日期': date_str,
    '時間': time_str,
    '品項': item,
    '金額': amount,
    '種類':category,
}])

# 使用 concat() 將新資料合併到舊的 df 中
df = pd.concat([df, new_data], ignore_index=True)

In [72]:
df

,日期,時間,品項,金額,種類
0,2025/4/19,9:45,麵包,50,NaN
1,2025-04-19,9:45,麵包,50,NaN
2,2025/4/19,9:45,麵包,50,NaN
3,2025-04-19,9:45,麵包,50,NaN
4,2023-05-03,9:34,蛋餅,30,NaN
5,2023-03-10,08:23,餅乾,45.0,娛樂


In [73]:
def save_to_google_sheets(new_data):
    """將資料儲存到 Google Sheets"""
    try:
        # 設定 Google Sheets 連接
        gsheets = setup_google_sheets()
        if gsheets is None:
            return False

        # 取得工作表物件
        worksheet = gsheets.worksheet('工作表1')

        # 將新資料轉換成列表格式
        data_to_write = new_data.values.tolist()

        # 將資料寫入工作表
        worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')

        print("資料已成功寫入 Google 工作表！")
        return True

    except Exception as e:
        print(f"寫入 Google 工作表時發生錯誤: {e}")
        return False
